In [1]:
# setup
import os
os.environ["KERAS_BACKEND"] = "torch"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import keras

if keras.backend.backend() == "torch":
    import torch
    print("Use torch backend")
    torch.autograd.set_grad_enabled(False)

import sys
# current_dir = os.path.dirname(os.path.abspath(__file__))
sys.path.append(os.path.join("../"))
sys.path.append(os.path.join("../../"))

import bayesflow as bf

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Use torch backend


/opt/miniconda3/envs/bayesflow/lib/python3.11/site-packages/keras/src/backend/torch/numpy.py:870: UserWarning: The operator 'aten::logspace.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1715184405838/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  torch.logspace(


In [2]:
from bayesflow.simulators.simulator import Simulator
# from bayesflow.types import Shape, Tensor
from torch import Tensor
from torch.distributions import Distribution
import torch.nn as nn

In [31]:
from custom_simulators import GenericSimulator, LikelihoodBasedModel, ParameterMask, Prior, RandomNumObs
from design_networks import RandomDesign, DeepAdaptiveDesign, EmitterNetwork, EncoderNetwork
from design_loss import NestedMonteCarlo

In [32]:
from experiment_1 import PolynomialRegression, PriorPolynomialReg
from custom_dataset import DataSet

In [33]:
inference_network = bf.networks.FlowMatching() # bf.networks.CouplingFlow()
summary_network = bf.networks.DeepSet(summary_dim = 10)

approximator = bf.Approximator(
        inference_network = inference_network,
        summary_network = summary_network,
        inference_variables = ["params"],
        inference_conditions = ["masks", "n_obs"],
        summary_variables = ["outcomes", "designs"])

In [35]:
design_shape = torch.Size([1])
mask_sampler = ParameterMask()
prior_sampler = PriorPolynomialReg()
T = 5
random_num_obs = RandomNumObs(min_obs = 1, max_obs = T)
random_design_generator = RandomDesign(design_shape = design_shape, scaler = 5)

model_1 = PolynomialRegression(mask_sampler = mask_sampler,
                                        prior_sampler = prior_sampler,
                                        tau_sampler = random_num_obs,
                                        design_generator = random_design_generator,
                                        sim_vars = {"sigma": 1.0})

encoder_net = EncoderNetwork(xi_dim = 1, y_dim = 1, hidden_dim = 128, encoding_dim = 10)
decoder_net = EmitterNetwork(input_dim = 10, hidden_dim = 2, output_dim = 1, scaler = 5) # [B, summary_dim] -> [B, xi_dim]
design_net = DeepAdaptiveDesign(encoder_net = encoder_net,
                                decoder_net = decoder_net,
                                design_shape = design_shape, 
                                summary_variables=["outcomes", "designs"])

model_2 = PolynomialRegression(mask_sampler = mask_sampler,
                                        prior_sampler = prior_sampler,
                                        tau_sampler = random_num_obs,
                                        design_generator = design_net,
                                        sim_vars = {"sigma": 1.0})

In [36]:
B = 64
dataset = DataSet(batch_shape = torch.Size([B]), 
                    joint_model_1 = model_1,
                    joint_model_2 = model_2)

In [47]:
dataset.set_stage(2)

In [48]:
test_sims = dataset.__getitem__(0)

In [24]:
masks = test_sims["masks"]
params = test_sims["params"]
outcomes = test_sims["outcomes"]
designs = test_sims["designs"]
n_obs = test_sims["n_obs"]**2

In [28]:
print(f"Shape of parameter mask {masks.shape}") # [B, model_dim]|
print(f"Shape of parameters {params.shape}") # [B, param_dim]
print(f"Shape of outcomes {outcomes.shape}") # [B, tau, y_dim]
print(f"Shape of designs {designs.shape}") # [B, tau, xi_dim]
print(n_obs[0].item()) # tau

Shape of parameter mask torch.Size([64, 4])
Shape of parameters torch.Size([64, 4])
Shape of outcomes torch.Size([64, 4, 1])
Shape of designs torch.Size([64, 4, 1])
4.0


In [29]:
approximator.compile(optimizer="AdamW")

In [30]:
approximator.fit(dataset, epochs=1, steps_per_epoch=10)

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 691ms/step - inference/loss: 4.5827 - loss: 4.5827 - summary/loss: 0.0000e+00


In [59]:
num_test_samples = 64
num_posterior_samples = 500

test_sims = dataset.__getitem__(0)
samples = approximator.sample(batch_shape=(num_test_samples, num_posterior_samples), data=test_sims)

In [61]:
print(samples.keys())

dict_keys(['params', 'summaries'])


In [63]:
print(samples['params'].shape) # [num_test_samples, num_posterior_samples, param_dim]

torch.Size([64, 500, 4])


In [29]:
obs_data = {"designs": out["designs"], "outcomes": out["outcomes"], "masks": out["masks"], "n_obs": out["n_obs"]}
# post_given_obs  = approximator.sample(batch_shape = (1, num_posterior_samples), data = obs_data)

In [31]:
obs_data["outcomes"].shape

torch.Size([1, 6, 4, 1])

In [25]:
obs_data["masks"].shape

torch.Size([1, 4])

In [26]:
obs_data = {"designs": out["designs"], "outcomes": out["outcomes"]}

In [36]:
obs_data.values

<function dict.values>

In [27]:
obs_data["outcomes"].shape

torch.Size([1, 9, 1])

In [28]:
from bayesflow.utils import filter_concatenate

filter_concatenate(obs_data, keys=["outcomes", "designs"]).shape

torch.Size([1, 9, 2])

In [29]:
summary_out = summary_network(filter_concatenate(obs_data, keys=["outcomes", "designs"]))

In [29]:
decoder_net = EmitterNetwork(input_dim = 10, hidden_dim = 24, output_dim = 1)

In [30]:
designs_net = DeepAdaptiveDesign(encoder_net=summary_network,
                                 decoder_net = decoder_net,
                                 design_shape = torch.Size([1]), 
                                 summary_variables=["outcomes", "designs"])

In [31]:
summary_batched = torch.randn(64, 10)

In [32]:
summary_batched.shape

torch.Size([64, 10])

In [33]:
decoder_net(summary_batched).shape

torch.Size([64, 1])

In [34]:
tensor = torch.empty((0, 1, 1))

In [36]:
tensor.shape[0]

0

In [40]:
xi = torch.tensor([1])

# Adding two singleton dimensions and repeating
B = 3  # Example size for repetition
expanded_xi = xi.view(1, 1, 1).repeat(B, 1, 1)

print(expanded_xi)

tensor([[[1]],

        [[1]],

        [[1]]])


In [41]:
expanded_xi.shape

torch.Size([3, 1, 1])